### Importieren der nötigen Bibliotheken

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

### Importieren der csv Datei und erstellen des Dataframes

In [ ]:
try:
    df = pd.read_csv('Online Shop Customer Sales Data.csv')

    print("Datensatz erfolgreich geladen!")
    
except FileNotFoundError:
    print("Fehler: 'Online Retail.csv' wurde im aktuellen Verzeichnis nicht gefunden.")
    print("Bitte stelle sicher, dass die Datei im selben Ordner wie dieses Notebook liegt oder gib den korrekten Pfad an.")
    print(f"Aktuelles Verzeichnis: {os.getcwd()}")
except Exception as e:
    print(f"Ein unerwarteter Fehler ist beim Laden der CSV-Datei aufgetreten: {e}")

 

Ein unerwarteter Fehler ist beim Laden der CSV-Datei aufgetreten: 'RangeIndex' object is not callable


Feature Engineering. Folgende Features überlegt: Agebuckets, RevenueBuckets, HVCustomer (High Value), AgeAvgRevenue, AOV (Average Order Value),SeasonLP(Last Purchase), RevenuePM (per Minute)

In [17]:
df

,Customer_id,Age,Gender,Revenue_Total,N_Purchases,Purchase_DATE,Purchase_VALUE,Pay_Method,Time_Spent,Browser,Newsletter,Voucher
0,504308,53,0,45.3,2,22.06.21,24.915,1,885,0,0,0
1,504309,18,1,36.2,3,10.12.21,2.896,2,656,0,0,1
2,504310,52,1,10.6,1,14.03.21,10.600,0,761,0,1,0
3,504311,29,0,54.1,5,25.10.21,43.280,1,906,0,1,0
4,504312,21,1,56.9,1,14.09.21,56.900,1,605,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
65791,570099,30,1,10.9,4,15.02.21,6.213,1,894,0,0,0
65792,570100,33,0,29.3,1,05.02.21,29.300,0,722,0,0,0
65793,570101,50,0,25.4,5,29.10.21,23.114,3,424,0,0,0
65794,570102,56,0,29.2,1,09.12.21,29.200,3,731,0,0,0


In [72]:
feature_engineering = pd.DataFrame()


#Age Bucket (0 = 16-25 ; 1 = 25-40; 2 = 40-63)
binlist =[16,25,40,63]
labellist = [0,1,2]
feature_engineering['Age_Buckets'] = pd.cut(df["Age"], bins=binlist,labels=labellist)

#AOV
feature_engineering['AOV']=df['Revenue_Total']/df['N_Purchases']

#HVCustomer
p_90 = df['Revenue_Total'].quantile(0.90)
binlist = [0,p_90-0.00001,max(df['Revenue_Total'])]
labellist = [0,1]
feature_engineering['HV_Customer'] = pd.cut(df['Revenue_Total'],bins=binlist,labels=labellist)
feature_engineering



,Age_Buckets,AOV,HV_Customer
0,2,22.650000,0
1,0,12.066667,0
2,2,10.600000,0
3,1,10.820000,1
4,0,56.900000,1
...,...,...,...
65791,1,2.725000,0
65792,1,29.300000,0
65793,2,5.080000,0
65794,2,29.200000,0


### Graphische Darstellung der neuen Features

In [75]:
age_group_counts = feature_engineering['Age_Buckets'].value_counts().sort_index().reset_index()
age_group_counts.columns = ['Altersgruppe', 'Anzahl der Kunden']

fig1 = px.bar(
    age_group_counts,
    x='Altersgruppe',
    y='Anzahl der Kunden',
    title='Verteilung der Altersgruppen',
    labels={'Altersgruppe': 'Altersgruppe', 'Anzahl der Kunden': 'Anzahl der Kunden'},
    color_discrete_sequence=px.colors.qualitative.Pastel # Optional: Farbschema
)
fig1.show()

In [86]:
fig = px.violin(
    feature_engineering,
    x='Age_Buckets',
    y='AOV',
    title='Dichteverteilung des Average Order Value (AOV) nach Altersgruppe',
    labels={'Age_Buckets': 'Altersgruppe', 'AOV': 'Average Order Value (AOV)'},
    color='Age_Buckets',
    color_discrete_sequence=px.colors.qualitative.Set2,
    box=True,  # Optional: Fügt einen Mini-Boxplot in die Violinen ein
    points='outliers' # Optional: Zeigt einzelne Outlier-Punkte (oder 'all' für alle Punkte)
)

fig.show()